<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/binance_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binance用オーダー発行API

`Binance`クラスは、以下の機能を提供します。

* オーダーの発行・キャンセル（REST API)　KEYとSECRETの設定が必要です。

また通過ペアーの設定は`BinanceConfig`クラスに定義してあります。パラーメータを修正することで任意の通過ペアーに対応可能ですが、テスト完了して提供しているものは以下になります。
* `BinanceConfig.BTCUSDT` 本番用BTCUSDTペアー（現物）


## 本サンプルの機能

本サンプルでは以下を行います。

* オーダーの発行(Limit, Market)
* オーダーの検索
* オーダーのキャンセル
* 資産残高の確認

### 利用している関数
* オーダー発行
  * `limit_order(side, price, size)` リミットオーダーの発行
  * `market_order(side, size)`　マーケットオーダーの発行
* オーダー状況確認
  * `order_status`　オーダーステータスの確認（約定済みも含めた直近のもの）
  * `open_orders`　未約定一覧
* オーダーキャンセル
  * `cancel_order(order_id)`　指定したIDのオーダキャンセル
  * `cancel_all_orders()`　未約定のオーダーのキャンセル
* アカウント情報
  * `account`　アカウント情報（残高など）

## API KEYについて（注：まずはテストネットで試してください）
動作させるためにはAPI KEYが必要です。
ローカル環境では、以下の値を環境変数に指定してください。
* BINANCE_API_KEY   APIキー
* BINANCE_API_SERECT  シークレット

Google Clabでは、`userdata`に設定し、`os.enviromnemt`で環境変数に設定してください。（`userdata`はColabのページの左側ツールバーにある🔑マークをクリック）。

## 必要ライブラリのインストール

`rbot`がBotFrameWorkの本体です。必要なライブラリを合わせてインストールします。インストール後ランタイムの再起動が必要になる場合があります。

In [ ]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。そのためバージョン指定しています。
! pip install --upgrade rbot

In [ ]:
# 必要ライブラリのインストール。環境によっては要・不要があるので適宜修正してください。
! pip install json2html

## Python コード本体

### 必要ライブラリのインポート

In [ ]:
import rbot
# Binanceクラスは、BinanceのAPIを利用して、取引所へオーダ発行を行うクラス
from rbot import Binance

# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig

from rbot import init_log, init_debug_log
#init_debug_log()

print("rbot version =", rbot.__version__)

In [ ]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

### Binanceオブジェクトの生成

`Binance`の引数`production`に本番の場合は`True`、テストネットの場合は`False`を入れて生成します。

In [ ]:
# Google Colabの場合は、userdataからAPIキーを取得
# 通常は、環境変数に設定してください。
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    
    # Google Colabの場合は、APIキーを入力してください。
    import os
    
    os.environ['BINANCE_API_KEY'] = userdata.get("BINANCE_API_KEY")
    os.environ['BINANCE_API_SECRET'] = userdata.get("BINANCE_API_SECRET")


# Binanceクラスのインスタンスを生成. production=Falseでテストネットを利用します。
exchange  = Binance(production=False)

## オーダー発行テスト

### Limit order

In [ ]:
# 現在のアカウント情報を取り出す
account = exchange.account

HTML(json2html.convert(account.__str__()))

In [ ]:
# accountにはすべての通貨の残高が入っているので、BTCとUSDTの残高を表示する
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

In [ ]:
# 有効な価格でオーダーを出すために、市場価格を取得する

config = BinanceConfig.BTCUSDT

market = exchange.open_market(config)
market.start_market_stream()

#market.board


bid_edge, ask_edge = market.edge_price

print("bid_edge =", bid_edge)
print("ask_edge =", ask_edge)

In [ ]:
# もういちどedge_priceを取得すると、最新の価格が取得できる.
bid_edge, ask_edge = market.edge_price

print("bid_edge =", bid_edge)
print("ask_edge =", ask_edge)

In [ ]:
# オーダーを有効にするフラグをセットする（ディフォルトではオーダー機能はDisableされていてエラーになる）
exchange.enable_order_with_my_own_risk = True

# 板より５００ドル安いところへ0.0001BTCの買い注文を出す
buy_limit_order = exchange.limit_order(config, "BUY", bid_edge - 500, 0.001)

HTML(json2html.convert(buy_limit_order.__str__()))

In [ ]:
# 未約定のオーダーを表示する。
open_order = exchange.get_open_orders(config)

HTML(json2html.convert(open_order.__str__()))

In [ ]:
# 板より５００ドル高いところへ0.0001BTCの売り注文を出す
sell_limit_order = exchange.limit_order(config, "SELL", ask_edge + 500, 0.001)

HTML(json2html.convert(sell_limit_order.__str__()))

In [ ]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず（合計２つ）(BINANCE TESTNETでは最初の１つしか表示されない問題あり)
open_order = exchange.get_open_orders(config)

HTML(json2html.convert(open_order.__str__()))

In [ ]:
# 最新の残高を確認。注文中の資産がLockされているはず。
account = exchange.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

In [ ]:
# 買いオーダーをキャンセルする

id_to_cancel = buy_limit_order[0].order_id

print("キャンセルするオーダーID =", id_to_cancel)

exchange.cancel_order(config, id_to_cancel)

In [ ]:
# キャンセルしたオーダーがなくなっていることを確認する。
open_order = exchange.get_open_orders(config)

HTML(json2html.convert(open_order.__str__()))

In [ ]:
# 最新の残高を確認。キャンセルされた分の資産のLockが解放される。
account = exchange.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

In [ ]:
# 売りオーダーをキャンセルする
id_to_cancel = sell_limit_order[0].order_id

print("キャンセルするオーダーID =", id_to_cancel)

exchange.cancel_order(config, id_to_cancel)

In [ ]:
# キャンセルしたオーダーがなくなっていることを確認する。
open_order = exchange.get_open_orders(config)

HTML(json2html.convert(open_order.__str__()))

In [ ]:
# 最新の残高を確認。Lock が解放されていることを確認する。  
account = exchange.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

## Market order

In [ ]:
# market orderを出す

market_order = exchange.market_order(config, "BUY", 0.001)   # 0.001BTCを市場価格で買う
HTML(json2html.convert(market_order.__str__()))

In [ ]:
# 最新の残高を確認。約定した分だけBTCが増えて、USDTの残高が減っているはず。
account = exchange.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])